EDAF75 - notebook for lab 2
===========================

**Author:** Christian Söderberg



## EDAF75 - lab 2: Experimenting with your own database:PROPERTIES:



To begin, we need to enable SQL in our notebook:



In [3]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


If we use `jupysql` (instead of `ipython-sql`) we get a default limit of 10 rows for any query &#x2013; to turn this limit off we can write:



In [4]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

We can now load the lab database:



In [18]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints &#x2013; to do that, we run:



In [19]:
%%sql
PRAGMA foreign_keys=ON;

Running query in 'sqlite:///movies.sqlite'

++
||
++
++

Now write SQL code for the following tasks:

-   Show the names of all movies.



In [20]:
%%sql
    SELECT title
    FROM movies

Running query in 'sqlite:///movies.sqlite'

title
The Dark Knight
Forrest Gump
Inception
The Lord of the Rings: The Fellowship of the Ring


-   Show the performance dates for one of the movies.



In [39]:
%%sql
SELECT imdb_id, theatre_name, date, start_time
FROM screenings
WHERE imdb_id = 'tt0468569'
ORDER BY date, start_time;

Running query in 'sqlite:///movies.sqlite'

imdb_id,theatre_name,date,start_time
tt0468569,SF Malmö,2026-02-16,18:00
tt0468569,Capitol Stockholm,2026-02-19,20:00


-   Show all data concerning performances at a given theatere on a given date.



In [44]:
%%sql
SELECT s.theatre_name, s.date,s.start_time, m.title
FROM screenings s
JOIN movies m ON s.imdb_id = m.imdb_id
WHERE s.theatre_name = 'SF Malmö'
  AND s.date = '2026-02-16'
ORDER BY s.start_time;

Running query in 'sqlite:///movies.sqlite'

theatre_name,date,start_time,title
SF Malmö,2026-02-16,15:00,Forrest Gump
SF Malmö,2026-02-16,18:00,The Dark Knight


-   List all customers



In [40]:
%%sql
SELECT full_name
FROM customers

Running query in 'sqlite:///movies.sqlite'

full_name
Anna Svensson
John Smith
Lisa Karlsson
Omar Ali


In [45]:
%%sql
INSERT INTO tickets(screening_id, username) VALUES
    (1, 'anna92'),
    (1, 'jsmith'),
    (2, 'lisa_k'),
    (2, 'omar_a'),
    (3, 'anna92'),
    (4, 'jsmith'),
    (5, 'lisa_k'),
    (6, 'omar_a');

Running query in 'sqlite:///movies.sqlite'

8 rows affected.

++
||
++
++

-   List all tickets



In [46]:
%%sql
SELECT *
FROM tickets

Running query in 'sqlite:///movies.sqlite'

ticket_uuid,screening_id,username
469e776686b30a2f192911eb2ada9dca,1,anna92
7944a14c4eaf6537c0b0aef8cbb8e445,1,jsmith
c29cbe1915a5e121e799e4dabe49ee0b,2,lisa_k
fffe7673e27f21b2032fcf179a1e48ce,2,omar_a
3d17b1723e8e36266e03aa0e935ccae1,3,anna92
dde555c30d2ae3353aeced6fa8a8e3cb,4,jsmith
de9bf3ccbccb7e66756cc20b99d67ad9,5,lisa_k
1ac8daf37bd5c8e7cba3b700880da0d9,6,omar_a


-   Create a new ticket to some performance (i.e., insert a new row in your table of tickets).



In [48]:
%%sql
INSERT INTO tickets(screening_id, username) VALUES
    (6, 'anna92')

Running query in 'sqlite:///movies.sqlite'

1 rows affected.

++
||
++
++

In newer versions of SQLite (since version 3.35, released in March 2021), and in [PostgreSQL]([https://www.postgresql.org/docs/current/sql-insert.html](https://www.postgresql.org/docs/current/sql-insert.html)), we can get any value generated during an insert using the `INSERT...-RETURNING` statement:

<pre>
<code>INSERT</code>
<code>INTO       students</code>
<code>VALUES     ('Amy', 3.9, 1200)</code>
<code>RETURNING  s&#95;id</code>
</pre>

which would return the generated `s_id` for the new student.

If your SQLite version is older than 3.35, and you can't upgrade, you can instead use the following idea: each row in a SQLite3 table has a `rowid` attribute, it is a unique integer which essentially tells in which order the rows were inserted, and it's not displayed in queries unless we ask for it. SQLite3 also have a function, `last_insert_rowid()`, which returns the `rowid` of the last inserted row of a table, so we can see the `s_id` of the most recently inserted student with the following query:

<pre>
<code>SELECT s&#95;id</code>
<code>FROM   students</code>
<code>WHERE  rowid = last&#95;insert&#95;rowid();</code>
</pre>

Now, check what ticket number we got for the ticket we created above (it should be the same as the ticket id, which should be a `randomblob`):



In [49]:
%%sql
SELECT *
FROM tickets
WHERE username = 'anna92'

Running query in 'sqlite:///movies.sqlite'

ticket_uuid,screening_id,username
469e776686b30a2f192911eb2ada9dca,1,anna92
3d17b1723e8e36266e03aa0e935ccae1,3,anna92
d495eb9d6f66bc91d58e202a6889f419,6,anna92


-   Try to insert two movie theaters with the same name (this
    should fail).



In [50]:
%%sql
INSERT INTO theatres(name, capacity) VALUES
    ('Bio Kinto', 250),
    ('Bio Kinto', 50),

Running query in 'sqlite:///movies.sqlite'

RuntimeError: (sqlite3.OperationalError) incomplete input
[SQL: INSERT INTO theatres(name, capacity) VALUES
    ('Bio Kinto', 250),
    ('Bio Kinto', 50),]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


-   Try to insert a performance where the theater doesn’t exist in the database (this should fail).



In [51]:
%%sql
INSERT INTO screenings(theatre_name, imdb_id, date, start_time) VALUES
    ('Franska biografen',        'tt0109830', '2026-02-16', '15:00'),

Running query in 'sqlite:///movies.sqlite'

RuntimeError: (sqlite3.OperationalError) incomplete input
[SQL: INSERT INTO screenings(theatre_name, imdb_id, date, start_time) VALUES
    ('Franska biografen',        'tt0109830', '2026-02-16', '15:00'),]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


-   Create a ticket where either the user or the performance doesn’t exist (this should fail).



In [52]:
%%sql
INSERT INTO tickets(screening_id, username) VALUES
    (6, 'kattencoco')

Running query in 'sqlite:///movies.sqlite'

RuntimeError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets(screening_id, username) VALUES
    (6, 'kattencoco')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
